In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob

import matplotlib.pyplot as plt
from itertools import chain
import tensorflow as tf
import cv2

In [ ]:
all_xray_df = pd.read_csv('./data/training_labels.csv')
all_image_paths = {os.path.basename(x): x for x in 
                   glob(os.path.join( './data/training', '*.png'))}
print('Scans found:', len(all_image_paths), ', Total Headers', all_xray_df.shape[0])
all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)
#all_xray_df.sample(3)

In [ ]:
all_xray_df['Label'] = all_xray_df['Finding Labels'].map(lambda x: 0 if x=='No Finding' else 1.0)
all_xray_df.shape

In [ ]:
from sklearn.utils  import shuffle


normal =  all_xray_df [ all_xray_df['Label'] == 0.0 ]
abnormal =  all_xray_df [ all_xray_df['Label'] == 1.0 ]

X_train = pd.concat([normal.sample(frac=0.8, random_state=0),\
                     abnormal.sample(frac=0.8,random_state=0)], axis=0)
X_valid = all_xray_df.loc[~all_xray_df.index.isin(X_train.index)]

X_train = shuffle(X_train)
X_valid = shuffle(X_valid)

# Data agumentation with train set
- flip
- random shift
- random affine

In [ ]:
import os
import glob

out_path = './data/agument/'

abnormal_df =  X_train [ X_train['Label'] == 1.0 ] 

if not os.path.exists(out_path):
    os.mkdir(out_path)

    for idx,row in abnormal_df.iterrows():
        img = cv2.imread(row['path'],cv2.IMREAD_GRAYSCALE)

        # flip     
        transform_img = cv2.flip(img,1)
        cv2.imwrite(out_path+'F'+os.path.basename(row['path']),transform_img)

        # random shift
        rows,cols = img.shape
        shift_var = np.random.randint(low=-6,high=6,size=2)
        M = np.float32([[1,0,shift_var[0]],[0,1,shift_var[1]]])
        transform_img = cv2.warpAffine(img,M,(cols,rows))
        cv2.imwrite(out_path+'S'+os.path.basename(row['path']),transform_img)

        # random affine
        pts1 = np.float32([[8,8],[58,58],[32,58]])
        shift_var = np.random.randint(low=-1,high=1,size=6).astype(np.float32) 
        pts2 = np.reshape(pts1.flatten()+shift_var,(-1,2))
        
        M = cv2.getAffineTransform(pts1,pts2)
        transform_img = cv2.warpAffine(img,M,(cols,rows))        
        cv2.imwrite(out_path+'A'+os.path.basename(row['path']),transform_img)


    
column_name = ['Image Index','Finding Labels','path','Label']
data_list = []
flist = glob.glob(out_path+'*.png')
for f in flist:
    path_name = f
    base_name = os.path.basename(f)
    data_list.append([base_name,'Effusion',path_name,1.0])

flip_df = pd.DataFrame(columns=column_name,data=data_list)


X_train = pd.concat([X_train,flip_df])
X_train.reset_index(inplace=True)


In [ ]:
X_train.groupby('Label').size()

# Over sample

In [ ]:
# Over sample
# train_over = pd.concat([X_train, X_train[X_train['Label'] == 1].sample(300) ])
# train_over = shuffle(train_over)

# Feature Standardization

In [ ]:


#train_x = train_over['path'].values
#train_y = train_over['Label'].values

train_x = X_train['path'].values
train_y = X_train['Label'].values

train_image = []
for f in train_x:
    img = cv2.imread(f,cv2.IMREAD_GRAYSCALE)
    m,s = cv2.meanStdDev(img)
    std_img = (img- m)/(1.e-6 + s)
    
    train_image.append(std_img.reshape((64,64,1)))
    
train_label = np.column_stack([1-train_y,train_y])


valid_x = X_valid['path'].values
valid_y = X_valid['Label'].values

valid_image = []
for f in valid_x:
    img = cv2.imread(f,cv2.IMREAD_GRAYSCALE)
    m,s = cv2.meanStdDev(img)
    std_img = (img- m)/(1.e-6 + s)
    valid_image.append(std_img.reshape((64,64,1)))
    
valid_label = np.column_stack([1-valid_y,valid_y])

In [ ]:
len(train_x),len(valid_x)

# CNN Model

In [ ]:

tf.reset_default_graph()

images = tf.placeholder(tf.float32, [None, 64, 64, 1])
true_out = tf.placeholder(tf.float32, [ None, 2])
kprob = tf.placeholder_with_default(0.9,shape=())
train_mode = tf.placeholder_with_default(True,shape=())

xavi_init = tf.contrib.layers.xavier_initializer_conv2d()
w1 = tf.Variable(xavi_init(shape=[3,3,1,32]),name='conv1_w')
conv1 = tf.nn.conv2d(images, w1, strides=[1, 1, 1, 1], padding='SAME')
norm1 = tf.layers.batch_normalization(conv1,training=train_mode)
relu1 = tf.nn.relu(norm1)
relu1 = tf.nn.dropout(relu1,kprob)


w2 = tf.Variable(xavi_init(shape=[3,3,32,32]),name='conv2_w')
conv2 = tf.nn.conv2d(relu1, w2, strides=[1, 1, 1, 1], padding='SAME')
norm2 = tf.layers.batch_normalization(conv2,training=train_mode)
relu2 = tf.nn.relu(norm2)
relu2 = tf.nn.dropout(relu2,kprob)

# 32, 32
pool1 = tf.nn.max_pool(relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


w3 = tf.Variable(xavi_init(shape=[3,3,32,64]),name='conv3_w')
conv3 = tf.nn.conv2d(pool1, w3, strides=[1, 1, 1, 1], padding='SAME')
norm3 = tf.layers.batch_normalization(conv3,training=train_mode)
relu3 = tf.nn.relu(norm3)
relu3 = tf.nn.dropout(relu3,kprob)


w4 = tf.Variable(xavi_init(shape=[3,3,64,64]),name='conv4_w')
conv4 = tf.nn.conv2d(relu3, w4, strides=[1, 1, 1, 1], padding='SAME')
norm4 = tf.layers.batch_normalization(conv4,training=train_mode)
relu4 = tf.nn.relu(norm4)
relu4 = tf.nn.dropout(relu4,kprob)


w5 = tf.Variable(xavi_init(shape=[3,3,64,64]),name='conv5_w')
conv5 = tf.nn.conv2d(relu4, w5, strides=[1, 1, 1, 1], padding='SAME')
norm5 = tf.layers.batch_normalization(conv5,training=train_mode)
relu5 = tf.nn.relu(norm5)
relu5 = tf.nn.dropout(relu5,kprob)



# 16, 16 , 64
pool2 = tf.nn.max_pool(relu5,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

fc1_reshape = tf.reshape(pool2, [-1, 16384])
fc1_w = tf.Variable(xavi_init(shape=[16384,1024]),name='fc1_w')
fc1_n = tf.layers.batch_normalization(tf.matmul(fc1_reshape,fc1_w) ,training=train_mode)
fc1 = tf.nn.relu(fc1_n)
fc1 = tf.nn.dropout(fc1,kprob)



fc2_w = tf.Variable(xavi_init(shape=[1024,128]),name='fc2_w')
fc1_n = tf.layers.batch_normalization(tf.matmul(fc1,fc2_w) ,training=train_mode)
fc2 = tf.nn.relu(fc1_n)
fc2 = tf.nn.dropout(fc2,kprob)

fc3_w = tf.Variable(xavi_init(shape=[128,2]),name='fc3_w')
fc3_b = tf.Variable(tf.zeros(shape=[2]),name='fc3_b')
logit = tf.nn.bias_add( tf.matmul(fc2,fc3_w) , fc3_b)




# cost function , accuracy

In [ ]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=true_out,logits=logit)
total_loss = tf.reduce_mean(cross_entropy)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.control_dependencies(update_ops):
    train_op = tf.train.AdamOptimizer(0.001).minimize(total_loss)

#train_op = tf.train.AdamOptimizer(0.001).minimize(total_loss)

correct_prediction = tf.equal(tf.argmax(logit, 1),tf.argmax(true_out, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    

# Saver

In [ ]:
saver = tf.train.Saver()

CHECKPOINT_PATH = './ckpt/vgg_xray.ckpt'
CHECKPOINT_FILE = '%s.meta' % CHECKPOINT_PATH

In [ ]:
#batch_size = 500
batch_size = 50
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    
    if os.path.isfile(CHECKPOINT_FILE):
        saver.restore(sess, CHECKPOINT_PATH)
        print('Restoring values from %s...' % CHECKPOINT_PATH)
    else:
        sess.run(init_op)

    for epoch in range(350):
        
        epoch_loss =0.0

        for start in range(0,len(train_image),batch_size):

            end = min( start+batch_size ,len(train_image))
            image_iter = train_image[start:end]
            label_iter = train_label[start:end]

            _,train_loss = sess.run([train_op,total_loss], feed_dict={images: image_iter, true_out:label_iter})
            epoch_loss += train_loss
            if end == len(train_image) :
                #loss,accr,train_logit = sess.run([total_loss,accuracy,logit], feed_dict={images: image_iter,kprob:0.8,true_out:label_iter})
                #print(epoch,epoch_loss,accr)
                print(epoch,epoch_loss)

                
        
        tp= fn= fp= tn= vloss = 0.0
        
        for start in range(0,len(valid_image),batch_size):

            end = min( start+batch_size ,len(valid_image))

            image_iter = valid_image[start:end]
            label_iter = valid_label[start:end]
            
            val_loss,val_logit = sess.run([total_loss,logit], feed_dict={images: image_iter, true_out:label_iter,kprob:1.0,train_mode:False})
            
            
            
            y_true = np.argmax(label_iter,axis=1)
            y_pred = np.argmax(val_logit,axis=1) 

            true_index =np.where(y_true == 1)
            flase_index = np.where(y_true == 0)


            tp += np.sum(y_pred[true_index])
            fn += np.sum(y_pred[true_index] == 0)
            fp += np.sum(y_pred[flase_index])
            tn += np.sum(y_pred[flase_index] == 0)
            
            vloss += val_loss

        recall = tp/(tp+fn+1e-7)
        precision = tp/(tp+fp+1e-7)
        f1score = 2*(recall*precision)/(recall+precision+1e-7)


        print('VALID ----- EPOCH : {:.3f}, LOSS : {:.3f}'.format(epoch,vloss))
        print('RECALL :{:.3f} , PRECISION : {:.3f} , F1 SCORE :{:.3f}'.format(recall, precision, f1score ))
        print('TP={}, TN={}, FP={}, FN={}'.format(tp,tn,fp,fn))
        
    #saver.save(sess, CHECKPOINT_PATH)

# Test 

In [ ]:

X_test = glob.glob('./data/test/*.png')
test_image = []
for f in X_test:
    img = cv2.imread(f,cv2.IMREAD_GRAYSCALE)
    m,s = cv2.meanStdDev(img)
    std_img = (img- m)/(1.e-6 + s)
    
    test_image.append(std_img.reshape((64,64,1)))
    
print('test set' , np.shape(test_image))

In [ ]:
batch_size = 50

with tf.Session() as sess:
    
    if os.path.isfile(CHECKPOINT_FILE):
        saver.restore(sess, CHECKPOINT_PATH)
        print('Restoring values from %s...' % CHECKPOINT_PATH)
               
        
        logit_list = []
        
        for start in range(0,len(test_image),batch_size):

            end = start + batch_size  if start + batch_size  < len(test_image)  else  len(test_image)

            image_iter = test_image[start:end]        
            test_logit = sess.run([logit], feed_dict={images: image_iter,kprob:1.0,train_mode:False})
            logit_list.append(test_logit)
            
        print('----done---')

In [ ]:
pred_test = np.row_stack([ i[0] for i in logit_list])
pred_label = np.argmax(pred_test,axis=1)

In [ ]:
test_data = {'path':X_test,'label':pred_label}
test_df = pd.DataFrame(test_data)